In [159]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

PREFIX='a video of '

In [ ]:
def check_precision(frames_number, video_name, predicted_events, event, anomaly_classes, prompts, mode):
    # TN Prediction and Reality are true
    # TN Prediction and Reality are false
    # FN Prediction is false and Reality is true
    # FP Prediction is true and Reality is false
    normal_class = PREFIX + "a normal view (persons walking or standing)"
    all_classes = [normal_class] + anomaly_classes
    frames = np.load("/home/ubuntu/Database/ALL/GT/gt_ALL.npz")
    frames= frames[video_name]
    frames = np.append(frames, frames[-1])
    # Create a dictionary to convert class names to numeric indices
    class_to_idx = {cls: idx for idx, cls in enumerate(all_classes)}
    # Example: {"a normal view...": 0, "a person riding...": 1, ...}
    num_classes = len(all_classes)  # 14
    cm = np.zeros((num_classes, num_classes), dtype=int)
    prompts = [prompt.lower().split('.')[0] for prompt in prompts]
    # Save frames_number, predicted_events, and prompts into a numpy array
    fn_stored=0
    tn_stored=0
    for i in range(len(predicted_events)):
        # Get ground truth
        is_anomaly = frames[frames_number[i] - 1]  # 0 or 1
        # Determine true class using EVENT when anomaly exists
        true_class = event if is_anomaly == 1 else normal_class
        pred_class = predicted_events[i]
        if prompts[i] == "" and (mode!=1 or mode!=2):
            pass
        elif prompts[i] == "yes":
            pass
        elif prompts[i] == "no":
            continue
        else:
            if true_class == pred_class:
                pass
            else:
                continue
            #continue
        # Get predicted class
        
        # Convert to indices (skip if class not recognized)
        true_idx = class_to_idx.get(true_class, -1)
        pred_idx = class_to_idx.get(pred_class, -1)
        
        if true_idx != -1 and pred_idx != -1:
            cm[true_idx, pred_idx] += 1
    # Get the index of your event class
    event_idx = class_to_idx[event]
    

    # Calculate metrics ONLY for your event class
    tp = cm[event_idx, event_idx]  # TNs for event
    fp = np.sum(cm[:, event_idx]) - tp  # FPs (other classes predicted as event)
    fn =  np.sum(cm[event_idx, :]) - tp  # FNs (event misclassified as others)
    tn = np.sum(cm) - tp - fp - fn  # TNs
    fn += fn_stored
    tn += tn_stored
    return tp, fp, fn, tn

In [161]:
events = [
        "Riding",
        "Fighting",
        "Playing",
        "Running",
        'Lying',
        "Chasing",
        "Jumping",
        "Falling",
        "Guiding",
        "Stealing",
        "Littering",
        "Tripping",
        "Pickpockering",
    ]
description = [
        "a person riding a bicycle on the street",  # Added context
        "multiple people engaged in a physical fight",  # More specific than "fighting"
        "a group of people playing a sport together",  # Added "sport" for visual clarity
        "a person running",  # Added context
        "a person lying motionless on the ground",  # "Motionless" helps distinguish from falling
        "a person aggressively chasing another person",  # "Aggressively" adds distinction
        "a person jumping high in the air with both feet",  # More specific than just "jumping"
        "a person accidentally falling to the ground",  # "Accidentally" helps distinguish
        "a person gently guiding another person by the arm",  # Added detail
        "a person stealing other person",  # More specific than "stealing"
        "a person deliberately throwing garbage on the ground",  # "Deliberately" adds clarity
        "a person tripping over an obstacle",  # More descriptive
        "a person pickpocketing a wallet from someone's pocket",  # Very specific
    ]
description = [
        "a person riding a bicycle",
        "a certain number of persons fighting",
        "a group of persons playing",
        "a person running",
        "a person lying in the floor",
        "a person chasing other person",
        "a person jumping",
        "a person falling",
        "a person guiding other person",
        "a person stealing other person",
        "a person throwing trash in the floor",
        "a person tripping",
        "a person stealing other person's pocket",
    ]

In [162]:
description=[PREFIX+desc for desc in description]
events_description = {event: desc for event, desc in zip(events, description)}

In [163]:

rute="/home/ubuntu/Database/ALL/Videos"
rute_stored='/home/ubuntu/Tesis/Storage/CLIPOneClassOld/'
modes=[1]
modes=[2]
usage='CLIP'
i=0
df= pd.DataFrame(columns=['Name', 'Event', 'Mode', 'TP', 'FP', 'FN', 'TN'])
for mode in modes:
    for folder_number in range(len(events)):
        folder=f"{rute}/{events[folder_number]}/"
        files= os.listdir(folder)
        for file in files:
            stored_file=file.split('.')[0]
            desc=events_description[events[folder_number]]
            info_stored=np.load(f"{rute_stored}{stored_file}_{usage}_{mode}_{desc}.npy", allow_pickle=True)
            frames_number=info_stored[0]
            predicted_events=info_stored[1]
            prompts=info_stored[2]
            event= events_description[events[folder_number]]
            tp, fp,fn, tn=check_precision(frames_number, stored_file, predicted_events, event, description, prompts, mode)
            row = {
            'Name':stored_file, 'Event':event, 'Mode':mode, 'TP':tp, 'FP':fp, 'FN':fn, 'TN':tn
            }
            df =pd.concat([df, pd.DataFrame([row])], ignore_index=True)
print(df)
df.to_csv(f"{rute_stored}/DFS/ResultsCLIP.csv", index=False)

        Name                                              Event Mode  TP FP  \
0     000350               a video of a person riding a bicycle    2   3  0   
1     000289               a video of a person riding a bicycle    2  26  0   
2    4_076_1               a video of a person riding a bicycle    2   6  0   
3     000326               a video of a person riding a bicycle    2   0  0   
4     000313               a video of a person riding a bicycle    2   0  0   
..       ...                                                ...  ...  .. ..   
247  1_075_1  a video of a person stealing other person's po...    2   0  0   
248  4_086_1  a video of a person stealing other person's po...    2   0  0   
249  D158_13  a video of a person stealing other person's po...    2   0  0   
250  D149_07  a video of a person stealing other person's po...    2   0  0   
251  D149_03  a video of a person stealing other person's po...    2   1  0   

     FN  TN  
0    31   0  
1    28   0  
2    16  

In [164]:
df_sum = df.groupby(["Event", "Mode"])[
    ["TP", "FP", "FN", "TN"]
].sum().reset_index()

# Initialize precision, recall, accuracy with zeros
df_sum["Precision"] = 0.0
df_sum["Recall"] = 0.0
df_sum["Accuracy"] = 0.0

# Calculate only where the denominator is not zero
mask_prec = (df_sum["TP"] + df_sum["FP"]) != 0
df_sum.loc[mask_prec, "Precision"] = (
    df_sum.loc[mask_prec, "TP"] / (df_sum.loc[mask_prec, "TP"] + df_sum.loc[mask_prec, "FP"])
)

mask_recall = (df_sum["TP"] + df_sum["FN"]) != 0
df_sum.loc[mask_recall, "Recall"] = (
    df_sum.loc[mask_recall, "TP"] / (df_sum.loc[mask_recall, "TP"] + df_sum.loc[mask_recall, "FN"])
)

mask_acc = (df_sum["TP"] + df_sum["FP"] + df_sum["FN"] + df_sum["TN"]) != 0
df_sum.loc[mask_acc, "Accuracy"] = (
    (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "TN"]) /
    (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "FP"] + df_sum.loc[mask_acc, "FN"] + df_sum.loc[mask_acc, "TN"])
)
df_sum.drop(columns=['TP', 'FP', 'FN', 'TN'], inplace=True)
df_sum['F1'] = 2 * df_sum['Precision'].astype(float) * df_sum['Recall'].astype(float) / (df_sum['Precision'].astype(float) + df_sum['Recall'].astype(float))
df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
df_sum.drop(columns=['Event'], inplace=True)
df_sum=df_sum.groupby(["Mode"]).mean().reset_index()
mode_names = {
        0: "CLIP y Reglas",
        1: "CLIP y Reglas con MLLM",
        2: "CLIP y Reglas Una clase",
    }
df_sum.rename(index=mode_names, inplace=True)
print(df_sum)

               Mode Precision    Recall  Accuracy        F1
CLIP y Reglas     2  0.691964  0.164518  0.390096  0.243335


/tmp/ipykernel_66423/2313330585.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0 0.9266770670826833 1.0 1.0 1.0 1.0 1.0 0.4751082251082251 1.0 0.59375]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sum.loc[mask_prec, "Precision"] = (
/tmp/ipykernel_66423/2313330585.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.019850309144158803 0.4 0.5490196078431373 0.28205128205128205
 0.09798270893371758 0.1854475209272376 0.2902208201892745
 0.21425085407515862 0.0 0.02040816326530612 0.0794979079497908 0.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sum.loc[mask_recall, "Recall"] = (
/tmp/ipykernel_66423/2313330585.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a

In [165]:
def calculate_ap(precision, recall):
    # Sort by recall (ascending)
    sorted_indices = np.argsort(recall)
    precision = np.array(precision)[sorted_indices]
    recall = np.array(recall)[sorted_indices]

    # Pad with (0,0) and (1,0)
    precision = np.concatenate(([0], precision, [0]))
    recall = np.concatenate(([0], recall, [1]))

    # Compute AP as the area under the raw curve (no interpolation)
    ap = 0.0
    for i in range(1, len(recall)):
        delta_recall = recall[i] - recall[i - 1]
        ap += delta_recall * precision[i]

    return ap
df_check = df.copy()
# Initialize precision, recall, accuracy with zeros
df_check["Precision"] = 0.0
df_check["Recall"] = 0.0
df_check["Accuracy"] = 0.0

# Calculate only where the denominator is not zero
mask_prec = (df_check["TP"] + df_check["FP"]) != 0
df_check.loc[mask_prec, "Precision"] = (
    df_check.loc[mask_prec, "TP"] / (df_check.loc[mask_prec, "TP"] + df_check.loc[mask_prec, "FP"])
)

mask_recall = (df_check["TP"] + df_check["FN"]) != 0
df_check.loc[mask_recall, "Recall"] = (
    df_check.loc[mask_recall, "TP"] / (df_check.loc[mask_recall, "TP"] + df_check.loc[mask_recall, "FN"])
)

mask_acc = (df_check["TP"] + df_check["FP"] + df_check["FN"] + df_check["TN"]) != 0
df_check.loc[mask_acc, "Accuracy"] = (
    (df_check.loc[mask_acc, "TP"] + df_check.loc[mask_acc, "TN"]) /
    (df_check.loc[mask_acc, "TP"] + df_check.loc[mask_acc, "FP"] + df_check.loc[mask_acc, "FN"] + df_check.loc[mask_acc, "TN"])
)
df_check['F1'] = 2 * df_check['Precision'].astype(float) * df_check['Recall'].astype(float) / (df_check['Precision'].astype(float) + df_check['Recall'].astype(float))
df_check['F1'].fillna(0, inplace=True)  # Handle division by zero
print(df_check)
categories = df_check["Event"].unique()
print(categories)
# Separate rows by category
df_check.fillna(0, inplace=True)
category_dfs = {category: df_check[df_check["Event"] == category] for category in categories}

mAP_process = []
for i in range(len(categories)):
    df1 = category_dfs[categories[i]]
    #
    print(df1)
    grouped = df1.groupby("Mode")
    # ----------------------------------------------------------------------
    # Ejecución del código
    ap_values = {}
    for mode, group in grouped:
        precision = np.array(group["Precision"].values)
        recall = np.array(group["Recall"].values)
        #print("Mode:", mode, "\n")
        #print("Precision:", precision, "Recall:", recall)
        # Comenta la siguiente línea para verificar si el error es aquí
        ap = calculate_ap(precision, recall)
        ap_values[mode] = ap
    mean_values = grouped[["Precision", "Recall",  "F1"]].mean()
    print("Average Precision (AP) for each mode:")
    for mode, ap in ap_values.items():
        print(f"Mode {mode}: {ap:.4f}")
    mean_values["AP"] = [ap_values[mode] for mode in mean_values.index]
    mean_values = mean_values[["AP"]]
    mAP_process.append(mean_values)
    # Plot the results
    mode_names = {
        0: "CLIP y Reglas",
        1: "CLIP y Reglas con MLLM",
        2: "CLIP y Reglas Una clase",
    }
    mean_values.rename(index=mode_names, inplace=True)
    # plt.xlim(0.4)
# Calculate the mean Average Precision (mAP) for each mode
mAP_values = pd.concat(mAP_process).groupby(level=0).mean()
print(mAP_values)

        Name                                              Event Mode  TP FP  \
0     000350               a video of a person riding a bicycle    2   3  0   
1     000289               a video of a person riding a bicycle    2  26  0   
2    4_076_1               a video of a person riding a bicycle    2   6  0   
3     000326               a video of a person riding a bicycle    2   0  0   
4     000313               a video of a person riding a bicycle    2   0  0   
..       ...                                                ...  ...  .. ..   
247  1_075_1  a video of a person stealing other person's po...    2   0  0   
248  4_086_1  a video of a person stealing other person's po...    2   0  0   
249  D158_13  a video of a person stealing other person's po...    2   0  0   
250  D149_07  a video of a person stealing other person's po...    2   0  0   
251  D149_03  a video of a person stealing other person's po...    2   1  0   

     FN  TN Precision    Recall  Accuracy        F1

/tmp/ipykernel_66423/3173919043.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0
 1.0 1.0 0.8581560283687943 1.0 1.0 1.0 1.0 1.0 0.9719626168224299 1.0 1.0
 0.4666666666666667 1.0 0.0 0.5625 1.0 1.0 0.0 0.09405940594059406 1.0
 0.8085106382978723 0.5 0.4 0.75 0.4444444444444444 0.25
 0.38461538461538464 0.5 0.9 1.0 1.0 1.0 0.0 0.11475409836065574
 0.972972972972973 0.5 0.625 1.0 1.0 0.32 0.92 0.6923076923076923 1.0 1.0
 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 0.75 0.125 1.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_check.loc[mask_prec, "Precision"] = (
/tmp/ipykernel_66423/3173919043.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08823529411764706 0.48148148148148145 0.